In [1]:
import logging
logging.basicConfig(
    level=logging.ERROR,
    format="%(asctime)s - %(name)s - %(levelname)s -- l.%(lineno)d: %(message)s",
)

In [2]:
#system_prompt = "You are a university-level physics professor teaching a graduate course in Physics."
system_prompt = "You are kindergarden teacher who introduces elementary physics concetps to a group of 5 year old in a language that they can understand."
prompt = "Explain nuclear fusion in 5 sentences or less."

# Test GenAIModel class

In [3]:
from src.genai_model.genai_model import GenAIModel
from src.utils.list import flatten_list_of_lists

In [4]:
_model = GenAIModel(model_type="gemini-2.0", system_promt=system_prompt)
_model.list_of_models

['gemini/gemini-2.0-flash-thinking-exp-01-21',
 'openrouter/google/gemini-2.0-flash-thinking-exp:free',
 'gemini/gemini-2.0-flash-thinking-exp-1219',
 'openrouter/google/gemini-2.0-flash-thinking-exp-1219:free',
 'gemini/gemini-2.0-flash-exp',
 'openrouter/google/gemini-2.0-flash-exp:free']

In [5]:
model_large = GenAIModel(model_type="large", system_promt=system_prompt)
model_large.list_of_models

['gemini/gemini-exp-1206',
 'openrouter/google/gemini-exp-1206:free',
 'gemini/gemini-exp-1121',
 'openrouter/google/gemini-exp-1121:free',
 'gemini/gemini-exp-1114',
 'openrouter/google/gemini-exp-1114:free',
 'gemini/gemini-1.5-pro-002',
 'gemini/gemini-1.5-pro-exp-0827',
 'openrouter/meta-llama/llama-3.1-405b-instruct:free',
 'gemini/gemini-1.5-pro-001',
 'mistral/mistral-large-2411',
 'mistral/mistral-large-2407',
 'mistral/mistral-large-2402']

In [6]:
model_large.completion(user_prompt=prompt, parameters={"temperature": 0.5, "top_p": 0.5})

ModelResponse(id='chatcmpl-56298c6a-e67f-469b-b6e3-94b214b14fc6', created=1743111773, model='gemini-exp-1206', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content="Okay class, listen up!\n\nImagine you have two tiny, tiny LEGO bricks. If you push them together *really, really* hard, so hard they smoosh into one bigger brick! When they smoosh together like that, POP! They make lots and lots of energy, like sunshine. That's kind of like how our big Sun makes its light and heat way up high. It's like a super-duper powerful smoosh happening all the time!", role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=96, prompt_tokens=39, total_tokens=135, completion_tokens_details=None, prompt_tokens_details=None), vertex_ai_grounding_metadata=[], vertex_ai_safety_results=[], vertex_ai_citation_metadata=[])

# Test LiteLLM

In [7]:
from litellm import completion

In [8]:
list_of_models = {
    "Gemini": "gemini/gemini-1.5-pro-002",
    "OpenRouter": "openrouter/meta-llama/llama-3.1-405b-instruct:free",
    "Mistral": "mistral/mistral-large-2411",
    "Groq": "groq/llama3-70b-8192",
}

In [9]:
parameters = {"temperature": 0.5}

for provider, model in list_of_models.items():
    print("="*25)
    print(f"Using model {model} from {provider}:\n")
    try:
        response_l = completion(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}
            ],
            **parameters
        )
        print(response_l['choices'][0].message.content)
    except Exception as e:
        print(f"The model failed, raising the following exception: {e}")
    break

Using model gemini/gemini-1.5-pro-002 from Gemini:

Imagine tiny bouncing balls, smaller than you can even see!  Sometimes, two of these balls zoom together *really* fast and stick, making a bigger ball.  When they stick, they get a little bit hotter and give off some sparkle.  The sun does this with its tiny balls, making light and heat for us!  It's like a big, sparkly hug!



# Test fallback

In [10]:
ll_models = list(list_of_models.values())
response_l = completion(
    model=ll_models[-1],
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ],
    fallbacks=ll_models[:-1],
)
print(response_l['choices'][0].message.content)

2025-03-27 17:43:02,719 - asyncio - ERROR -- l.1785: Task was destroyed but it is pending!
task: <Task pending name='Task-9' coro=<Logging.async_success_handler() running at /usr/local/lib/python3.11/site-packages/litellm/litellm_core_utils/litellm_logging.py:1515>>


OH BOY, let's talk about STARS!

You know how we need to combine blocks to make a big tower? Well, inside stars, tiny things called atoms combine to make even more energy! It's like a super powerful hug that makes the star super bright and hot! This special hug is called nuclear fusion. And that's why stars shine so brightly in the sky!


/usr/local/lib/python3.11/asyncio/base_events.py:679: RuntimeWarning: coroutine 'Logging.async_success_handler' was never awaited
  self._ready.clear()


In [11]:
response_l

ModelResponse(id='chatcmpl-d2440e83-e703-43d7-8331-0c78729f7c9f', created=1743111782, model='groq/llama3-70b-8192', object='chat.completion', system_fingerprint='fp_2e0feca3c9', choices=[Choices(finish_reason='stop', index=0, message=Message(content="OH BOY, let's talk about STARS!\n\nYou know how we need to combine blocks to make a big tower? Well, inside stars, tiny things called atoms combine to make even more energy! It's like a super powerful hug that makes the star super bright and hot! This special hug is called nuclear fusion. And that's why stars shine so brightly in the sky!", role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=78, prompt_tokens=55, total_tokens=133, completion_tokens_details=None, prompt_tokens_details=None, queue_time=0.20339941, prompt_time=0.00136738, completion_time=0.315697004, total_time=0.317064384), x_groq={'id': 'req_01jqcrxzrhf8na6fg7c86mhknv'})

In [12]:
response_l['model']

'groq/llama3-70b-8192'

# Compare Gemini and Groq models

## Gemini

In [13]:
import os
import google.generativeai as genai
from groq import Groq


genai.configure(api_key=os.environ.get("GEMINI_API_KEY"))
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))

In [14]:
model_name = "gemini-1.5-flash"
param = {"temperature": 0.0, "top_p": 0.1}
prompt = "Describe OpenAI in a few sentences."

In [15]:
model = genai.GenerativeModel(model_name)
response_g = model.generate_content(
    prompt,
    generation_config = genai.GenerationConfig(**param)
)
response_g.text

'OpenAI is a leading artificial intelligence research company that develops and promotes friendly AI.  They create cutting-edge models like GPT, DALL-E, and others,  available through APIs and research publications, aiming to benefit humanity.  Their mission is to ensure that artificial general intelligence (AGI) benefits all of humankind.\n'

In [16]:
# https://docs.litellm.ai/docs/providers/gemini
response_l = completion(
    model=f"gemini/{model_name}",
    messages=[{"role": "user", "content": prompt}],
    **param
)
response_l['choices'][0].message.content

'OpenAI is a leading artificial intelligence research company that develops and promotes friendly AI.  They create cutting-edge models like GPT, DALL-E, and others,  with the goal of benefiting humanity.  Their work spans various AI fields, including natural language processing, computer vision, and reinforcement learning.\n'

### Check reproducibility in the case temperature=0.0 and top_p < 1.0
In the case top_p = 1.0, I would still get some stochasticity, probably since temperature=0.0 is approximated algorithmically (using eps)

In [17]:
model = genai.GenerativeModel(model_name)
response_g = model.generate_content(
    prompt,
    generation_config = genai.GenerationConfig(**param)
)
response_g.text

'OpenAI is a leading artificial intelligence research company that develops and promotes friendly AI.  They create cutting-edge models like GPT, DALL-E, and others,  available through APIs and research publications, aiming to benefit humanity.  Their mission is to ensure that artificial general intelligence (AGI) benefits all of humankind.\n'

In [18]:
model = genai.GenerativeModel(model_name)
response_g = model.generate_content(
    prompt,
    generation_config = genai.GenerationConfig(**param)
)
response_g.text

'OpenAI is a leading artificial intelligence research company that develops and promotes friendly AI.  They create cutting-edge models like GPT, DALL-E, and others,  available through APIs and research publications, aiming to benefit humanity.  Their mission is to ensure that artificial general intelligence (AGI) benefits all of humankind.\n'

In [19]:
response_l = completion(
    model=f"gemini/{model_name}",
    messages=[{"role": "user", "content": prompt}],
    **param
)
response_l['choices'][0].message.content

'OpenAI is a leading artificial intelligence research company that develops and promotes friendly AI.  They create cutting-edge models like GPT, DALL-E, and others,  available through APIs and research publications, aiming to benefit humanity.  Their mission is to ensure that artificial general intelligence (AGI) benefits all of humankind.\n'

## Compare Groq

In [20]:
model_name = "llama-3.3-70b-versatile"
param = {"temperature": 0.0, "top_p": 0.1}
prompt = "Describe OpenAI in a few sentences."

In [21]:
chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": prompt,}],
    model=model_name,
    **param
)
print(chat_completion.choices[0].message.content)

OpenAI is a leading artificial intelligence research organization that focuses on developing and promoting AI technologies that benefit humanity. Founded in 2015, OpenAI aims to create advanced AI systems that can learn, reason, and interact with humans in a safe and transparent manner. The organization is known for its development of various AI models, including language models like GPT, which can generate human-like text and converse with users in a natural way. OpenAI's mission is to ensure that the benefits of AI are shared by all people, and to prioritize the development of AI systems that are aligned with human values.


In [22]:
# https://docs.litellm.ai/docs/providers/groq
response_l = completion(
    model=f"groq/{model_name}",
    messages=[{"role": "user", "content": prompt}],
    **param
)
response_l['choices'][0].message.content

"OpenAI is a leading artificial intelligence research organization that focuses on developing and promoting AI technologies that benefit humanity. Founded in 2015, OpenAI aims to create advanced AI systems that can learn, reason, and interact with humans in a safe and transparent manner. The organization is known for its development of various AI models, including language models like GPT, which can generate human-like text and converse with users in a natural way. OpenAI's mission is to ensure that the benefits of AI are shared by all people, and to prioritize the development of AI systems that are aligned with human values."